<a href="https://colab.research.google.com/github/Nhlanhla-advocate/Nhlanhla-Advocate-repo/blob/main/Nhlanhla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
import cv2
import gradio as gr
import os
from transformers import pipeline
from PIL import Image

# Initialize the image captioning pipeline
captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")

def process_video(video_path, frame_skip=100):
    # Open the video file using OpenCV
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        return "Error: Could not open video."
    frame_captions_dict = {}
    frame_count = 0
    captions = []

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_skip == 0:
            try:
                # Convert the OpenCV frame (BGR) to a PIL image (RGB)
                image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                # Generate caption using the Hugging Face pipeline
                caption = captioner(image, max_new_tokens=50)[0]['generated_text']
                print(f"Caption generated for frame {frame_count}: {caption}")

                # Store caption in both the dictionary and list
                frame_captions_dict[frame_count] = caption
                #captions.append((frame_count, caption))

            except Exception as e:
                print(f"Error processing frame {frame_count}: {e}")
                continue

        frame_count += 1

    cap.release()  # Release the video capture object

    # Return the generated captions dictionary
    return frame_captions_dict

# Create the Gradio interface
iface = gr.Interface(
    fn=process_video,
    inputs=[
        gr.Video(label="Upload Video"),
        gr.Slider(1, 200, value=100, step=1, label="Frame Skip Rate (1 = Every Frame)")
    ],
    outputs=gr.JSON(label="Generated Captions"),
    title="Let's do something awesome",
    description="Upload a video."
)

# Launch the Gradio app
iface.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://09d226ff7bcf0ffa0b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
